In [1]:
!pip install mediapipe opencv-python

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp312-cp312-win_amd64.whl.metadata (1.4 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ml_dtypes-0.5.3-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.7.2-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.2-cp312-cp312-win_amd64.whl.metadata (1.4 kB)
  Using cached jax-0.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.7.1-cp312-cp312-win_amd64.whl.metadata (1.3 kB

In [5]:
import cv2
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

POSE_COLOR = (0, 255, 0)
LEFT_HAND_COLOR = (255, 0, 0)
RIGHT_HAND_COLOR = (0, 0, 255)

UPPER_COLOR = (255, 150, 0)    # ORANGE
LOWER_COLOR = (150, 0, 255)    # PURPLE

holistic = mp_holistic.Holistic(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

cap = cv2.VideoCapture(0)

cv2.namedWindow("Tracker", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Tracker", 1920, 1080)  # almost fullscreen but with title bar

def body_too_close(landmarks):
    if not landmarks:
        return True
    visible_count = sum(1 for lm in landmarks.landmark if lm.visibility > 0.4)
    return visible_count < 15

UPPER_BODY_CONNECTIONS = [
    (11, 12),  # Shoulders
    (11, 13), (13, 15),  # Left arm
    (12, 14), (14, 16),  # Right arm
    (11, 23), (12, 24)   # Upper torso
]

LOWER_BODY_CONNECTIONS = [
    (23, 24),  # Hips
    (23, 25), (25, 27),  # Left leg
    (24, 26), (26, 28)   # Right leg
]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = holistic.process(rgb)

    if result.pose_landmarks:
        lm = result.pose_landmarks.landmark

        mp_drawing.draw_landmarks(
            frame,
            result.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=POSE_COLOR, thickness=2)
        )

        for a, b in UPPER_BODY_CONNECTIONS:
            ax, ay = int(lm[a].x * frame.shape[1]), int(lm[a].y * frame.shape[0])
            bx, by = int(lm[b].x * frame.shape[1]), int(lm[b].y * frame.shape[0])
            cv2.line(frame, (ax, ay), (bx, by), UPPER_COLOR, 4)

        for a, b in LOWER_BODY_CONNECTIONS:
            ax, ay = int(lm[a].x * frame.shape[1]), int(lm[a].y * frame.shape[0])
            bx, by = int(lm[b].x * frame.shape[1]), int(lm[b].y * frame.shape[0])
            cv2.line(frame, (ax, ay), (bx, by), LOWER_COLOR, 4)

    if result.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            result.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=LEFT_HAND_COLOR, thickness=2, circle_radius=3)
        )

    if result.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            result.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=RIGHT_HAND_COLOR, thickness=2, circle_radius=3)
        )

    if body_too_close(result.pose_landmarks):
        cv2.putText(frame, "MOVE AWAY FROM CAMERA",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (0, 0, 255), 4)

    cv2.imshow("Tracker", frame)

    if cv2.getWindowProperty("Tracker", cv2.WND_PROP_VISIBLE) < 1:
        break

    key = cv2.waitKey(1)
    if key & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()
